In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, roc_auc_score
%matplotlib inline

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv('train.csv')

In [3]:
df_test = pd.read_csv('test.csv')

In [4]:
print df_train.shape
print df_test.shape

(101408, 2)
(188920, 1)


In [5]:
df_train.head(10)

,Word,Label
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0
5,Аарона,1
6,Аароне,0
7,Ааронов,0
8,Аахена,0
9,Абабков,1


In [6]:
df_test.head()

,Word
0,Аалто
1,ААР
2,Аара
3,Ааре
4,Аарон


BR

In [7]:
len(df_train[df_train.Label == 1])/(len(df_train)+0.0)

0.10490296623540549

Train/Test

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(df_train.Word,df_train.Label, random_state = 0, test_size = 0.3)

In [9]:
print Y_train.sum()/(len(Y_train)+0.0)
print len(Y_train)
print Y_test.sum()/(len(Y_test)+0.0)
print len(Y_test)


0.104796788054
70985
0.105150708346
30423


### Get n_gramms

In [10]:
vectorizer = CountVectorizer(analyzer ='char_wb',ngram_range =(1,8),encoding='utf8')

In [11]:
pd.concat([df_train.Word,df_test.Word]).shape

(290328L,)

In [12]:
vectorizer.fit(pd.concat([df_train.Word,df_test.Word]))

CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding='utf8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 8), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [13]:
X_train_cv = vectorizer.transform(X_train)

In [14]:
X_test_cv = vectorizer.transform(X_test)

In [15]:
print(X_train_cv.shape)
print(X_test_cv.shape)

(70985, 1145746)
(30423, 1145746)


In [16]:
X_train_cv

<70985x1145746 sparse matrix of type '<type 'numpy.int64'>'
	with 3777124 stored elements in Compressed Sparse Row format>

In [17]:
X_test_cv

<30423x1145746 sparse matrix of type '<type 'numpy.int64'>'
	with 1621102 stored elements in Compressed Sparse Row format>

In [18]:
vectorizer.get_feature_names()

[u' ',
 u' "',
 u' "\u0438',
 u' "\u0438\u0440',
 u' "\u0438\u0440\u043a',
 u' "\u0438\u0440\u043a\u0443',
 u' "\u0438\u0440\u043a\u0443\u0442',
 u' "\u0438\u0440\u043a\u0443\u0442 ',
 u' "\u0438\u0440\u043a\u0443\u0442"',
 u' -',
 u' - ',
 u' 7',
 u' 7 ',
 u' \xab',
 u' \xab\u0433',
 u' \xab\u0433\u0430',
 u' \xab\u0433\u0430\u0437',
 u' \xab\u0433\u0430\u0437\u043f',
 u' \xab\u0433\u0430\u0437\u043f\u0440',
 u' \xab\u0433\u0430\u0437\u043f\u0440\u043e',
 u' \xab\u043e',
 u' \xab\u043e\u0431',
 u' \xab\u043e\u0431\u043e',
 u' \xab\u043e\u0431\u043e\u0440',
 u' \xab\u043e\u0431\u043e\u0440\u043e',
 u' \xab\u043e\u0431\u043e\u0440\u043e\u043d',
 u' \xab\u0441',
 u' \xab\u0441\u0435',
 u' \xab\u0441\u0435\u0432',
 u' \xab\u0441\u0435\u0432\u043c',
 u' \xab\u0441\u0435\u0432\u043c\u0430',
 u' \xab\u0441\u0435\u0432\u043c\u0430\u0448',
 u' \xab\u0441\u043b',
 u' \xab\u0441\u043b\u0430',
 u' \xab\u0441\u043b\u0430\u0432',
 u' \xab\u0441\u043b\u0430\u0432\u043d',
 u' \xab\u0441\u043b\u0430\u

### Train/Test split и построение классификатора

In [19]:
clf = MultinomialNB()
clf.fit(X_train_cv,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
roc_auc_score(Y_test,clf.predict_proba(X_test_cv)[:,1])

0.89085882103731906

In [21]:
print (classification_report(clf.predict(X_test_cv),Y_test))

             precision    recall  f1-score   support

          0       0.99      0.93      0.96     28838
          1       0.40      0.80      0.53      1585

avg / total       0.96      0.93      0.94     30423



### Формирование файла с предсказаниями

In [24]:
df_test_cv = vectorizer.transform(df_test.Word)

In [27]:
type(df_test_cv)

scipy.sparse.csr.csr_matrix

In [25]:
df_test_cv.shape

(188920, 1145746)

In [33]:
pd.DataFrame(clf.predict_proba(df_test_cv)[:,1]).to_csv('Submit_khrenov.csv')